## финальное:

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import warnings
from shapely import wkt, Polygon, Point
warnings.simplefilter(action='ignore', category=FutureWarning)

/Users/dariaonikova/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_62733/778130093.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (h

## Оценим кластеризацию:

In [24]:
houses_geom = pd.read_csv("hdbscan.csv")

In [29]:
X = houses_geom.drop(columns=['cluster_label', 'CLUSTER_HDBSCAN'])

from sklearn.metrics import davies_bouldin_score, silhouette_score
from sklearn.cluster import KMeans

# Calculate the Davies-Bouldin Index
dbi = davies_bouldin_score(X, houses_geom['cluster_label'])
print("KMEANS - Davies-Bouldin Index:", dbi)
silhouette_avg = silhouette_score(X, houses_geom['cluster_label'])
print("KMEANS - Silhouette Score:", silhouette_avg)

dbi = davies_bouldin_score(X, houses_geom['CLUSTER_HDBSCAN'])
print("HDBSCAN - Davies-Bouldin Index:", dbi)
silhouette_avg = silhouette_score(X, houses_geom['CLUSTER_HDBSCAN'])
print("HDBSCAN - Silhouette Score:", silhouette_avg)

KMEANS - Davies-Bouldin Index: 0.41187054135917367
KMEANS - Silhouette Score: 0.6514462933155907
HDBSCAN - Davies-Bouldin Index: 3.8687133361377914
HDBSCAN - Silhouette Score: 0.6707560610554963


# COUNT

In [ ]:
houses_geom = pd.read_csv("hdbscan.csv")

In [31]:
areas = pd.read_csv('areas-pop-geom-space.csv')
areas['hs_ratio'] = areas['living_space'] / areas['population']
areas = areas.drop(columns=['field_1'])
areas['geometry'] = areas['geometry'].apply(wkt.loads)

areas

,district_code,area_name,population,geometry,space,living_space,hs_ratio
0,zo,Внуково,24687,"POLYGON ((37.2458 55.6019109, 37.2418516 55.59...",378945.00,169846.50,6.879998
1,zo,Можайский,137634,"POLYGON ((37.3307568 55.685325, 37.3294276 55....",3525138.10,2951192.00,21.442318
2,svo,Бибирево,158939,"POLYGON ((37.6440665 55.8976996, 37.6356092 55...",2572168.20,2430648.30,15.292963
3,vo,Богородское,110049,"POLYGON ((37.6874473 55.8167675, 37.6878233 55...",2338132.30,2171861.02,19.735400
4,uo,Братеево,108582,"POLYGON ((37.7295024 55.6408925, 37.7453811 55...",1844167.50,1758697.07,16.196949
...,...,...,...,...,...,...,...
87,tso,Красносельский,47865,"POLYGON ((37.684842 55.7808113, 37.684818 55.7...",1209627.35,855932.54,17.882222
88,tso,Мещанский,60044,"POLYGON ((37.620909 55.7589912, 37.6215936 55....",1788118.70,1274209.79,21.221268
89,tso,Пресненский,128784,"POLYGON ((37.5812267 55.7745578, 37.5814805 55...",4685420.60,3314867.87,25.739749
90,tso,Таганский,123382,"POLYGON ((37.6352868 55.7538551, 37.6352315 55...",3213656.90,2499755.06,20.260290


In [32]:
areas_geom = gpd.GeoDataFrame(
    areas, geometry=areas.geometry)
areas_geom.head()

,district_code,area_name,population,geometry,space,living_space,hs_ratio
0,zo,Внуково,24687,"POLYGON ((37.24580 55.60191, 37.24185 55.59705...",378945.0,169846.50,6.879998
1,zo,Можайский,137634,"POLYGON ((37.33076 55.68532, 37.32943 55.68579...",3525138.1,2951192.00,21.442318
2,svo,Бибирево,158939,"POLYGON ((37.64407 55.89770, 37.63561 55.89368...",2572168.2,2430648.30,15.292963
3,vo,Богородское,110049,"POLYGON ((37.68745 55.81677, 37.68782 55.81691...",2338132.3,2171861.02,19.735400
4,uo,Братеево,108582,"POLYGON ((37.72950 55.64089, 37.74538 55.63990...",1844167.5,1758697.07,16.196949


In [10]:
head_dict = {
    "Адрес ОЖФ": "addr",
    "Глобальный уникальный идентификатор дома по ФИАС": "fias_code",
    "Способ управления": "lead_type",
    "Наименование организации, осуществляющей управление домом": "lead_name",
    "Тип дома" : "type",
    "Состояние": "cond",
    "Общая площадь дома": "space",
    "Жилая площадь в доме": "living_space",
    "Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития" : "dorm_federal",
    "Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития" : "dorm_municipal",
    "Тип общежития" : "dorm_type",
    "Тип помещения (блока)": "room_type",
    "Дата сноса объекта" : "demolition_date",
    "geometry" : "geometry",
    "lat": "lat",
    "lon": "lon",
    "area_num": "area_num",
}

In [12]:

import os


df_list = []
houses_dict = dict()
houses_real_order = dict()
i = 0

for areas_geom_index, areas_geom_row in areas_geom.iterrows():
    area_name = areas_geom_row['area_name']
    houses_dict[area_name] = areas_geom_index
    houses_real_order[area_name] = i
    print(areas_geom_index, area_name)

    path = os.path.join('mow-reestr-areas', f'{area_name}.csv')
    df = pd.read_csv(path)
    df_list.append(df)
    i += 1

houses = pd.concat(df_list, ignore_index=True)



0 Внуково
1 Можайский
2 Бибирево
3 Богородское
4 Братеево
5 Коптево
6 Крылатское
7 Некрасовка
8 Проспект Вернадского
9 Ростокино
10 Свиблово
11 Северный
12 Сокольники
13 Академический
14 Алексеевский
15 Алтуфьевский
16 Бабушкинский
17 Бутырский
18 Войковский
19 Головинский
20 Лосиноостровский
21 Молжаниновский
22 Нагорный
23 Нижегородский
24 Ново-Переделкино
25 Обручевский
26 Беговой
27 Бирюлёво Восточное
28 Бирюлёво Западное
29 Вешняки
30 Восточное Дегунино
31 Дорогомилово
32 Зюзино
33 Капотня
34 Косино-Ухтомский
35 Котловка
36 Кузьминки
37 Куркино
38 Лефортово
39 Лианозово
40 Люблино
41 Марьино
42 Метрогородок
43 Митино
44 Нагатинский Затон
45 Новогиреево
46 Новокосино
47 Очаково-Матвеевское
48 Перово
49 Печатники
50 Покровское-Стрешнево
51 Раменки
52 Северное Бутово
53 Северное Медведково
54 Северное Тушино
55 Сокол
56 Солнцево
57 Строгино
58 Текстильщики
59 Тёплый Стан
60 Фили-Давыдково
61 Хамовники
62 Хорошёво-Мнёвники
63 Царицыно
64 Чертаново Северное
65 Чертаново Центральное
66 

In [13]:
i = 0
for area_name in houses_real_order:
    houses_real_order[area_name] = i
    i += 1
houses_real_order

{'Внуково': 0,
 'Можайский': 1,
 'Бибирево': 2,
 'Богородское': 3,
 'Братеево': 4,
 'Коптево': 5,
 'Крылатское': 6,
 'Некрасовка': 7,
 'Проспект Вернадского': 8,
 'Ростокино': 9,
 'Свиблово': 10,
 'Северный': 11,
 'Сокольники': 12,
 'Академический': 13,
 'Алексеевский': 14,
 'Алтуфьевский': 15,
 'Бабушкинский': 16,
 'Бутырский': 17,
 'Войковский': 18,
 'Головинский': 19,
 'Лосиноостровский': 20,
 'Молжаниновский': 21,
 'Нагорный': 22,
 'Нижегородский': 23,
 'Ново-Переделкино': 24,
 'Обручевский': 25,
 'Беговой': 26,
 'Бирюлёво Восточное': 27,
 'Бирюлёво Западное': 28,
 'Вешняки': 29,
 'Восточное Дегунино': 30,
 'Дорогомилово': 31,
 'Зюзино': 32,
 'Капотня': 33,
 'Косино-Ухтомский': 34,
 'Котловка': 35,
 'Кузьминки': 36,
 'Куркино': 37,
 'Лефортово': 38,
 'Лианозово': 39,
 'Люблино': 40,
 'Марьино': 41,
 'Метрогородок': 42,
 'Митино': 43,
 'Нагатинский Затон': 44,
 'Новогиреево': 45,
 'Новокосино': 46,
 'Очаково-Матвеевское': 47,
 'Перово': 48,
 'Печатники': 49,
 'Покровское-Стрешнево':

In [14]:
houses = houses[head_dict.keys()]
houses = houses.rename(columns=head_dict)
houses

,addr,fias_code,lead_type,lead_name,type,cond,space,living_space,dorm_federal,dorm_municipal,dorm_type,room_type,demolition_date,geometry,lat,lon,area_num
0,"108809, Москва г, д. Толстопальцево, ул. Совет...",e2b601d1-9dce-4daa-93c2-633ee59f7f5c,Не выбран,NaN,Жилой,Исправный,90.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
1,"108809, Москва г, д. Толстопальцево, ул. Совет...",1389f7e1-1948-43c5-a769-944efdf1777d,Не выбран,NaN,Жилой,Исправный,211.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
2,"108809, Москва г, д. Толстопальцево, ул. Совет...",1bd548bd-d46a-4991-9062-57eeb638a2ce,Не выбран,NaN,Жилой,Исправный,113.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
3,"108809, Москва г, д. Толстопальцево, ул. Совет...",ac2010c3-0c83-47e6-b67e-ea555f181975,Не выбран,NaN,Жилой,Исправный,275.0,NaN,-,-,-,NaN,NaN,POINT (37.1928963 55.6036271),55.603627,37.192896,0
4,"108809, Москва г, д. Толстопальцево, ул. Совет...",55cc8c65-03a7-46f2-ae61-e8a6e3420b25,Не выбран,NaN,Жилой,Исправный,69.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66795,"127055, Москва г, ул. Тихвинская, д. 39",adf12acf-cfca-4b7e-acab-af962a58a5de,УО,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТСЖ ...",Многоквартирный,Исправный,23595.3,16519.0,нет,нет,-,NaN,NaN,POINT (37.598968 55.791289),55.791289,37.598968,97
66796,"127055, Москва г, ул. Тихвинская, д. 3, корп. 1",01a3cfd9-65de-4712-b305-a8e9420a5576,УО,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,13540.5,11495.0,-,-,-,NaN,NaN,POINT (37.600882 55.785939),55.785939,37.600882,97
66797,"127055, Москва г, ул. Тихвинская, д. 7",7136e1b0-e4a6-44aa-a94d-ed4f956f561c,УО,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2652.6,1948.0,-,-,-,NaN,NaN,POINT (37.600783 55.786779),55.786779,37.600783,97
66798,"127055, Москва г, ул. Тихвинская, д. 9",7ade45e2-1084-4c3a-8cc3-c4abc3129173,УО,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,5328.6,1241.0,-,-,-,NaN,NaN,POINT (37.600352 55.787275),55.787275,37.600352,97


In [15]:
houses.dropna(subset = ['lon', 'lat'], inplace=True)
# houses = houses.drop_duplicates(subset=['addr'])

In [16]:
houses.shape

(66800, 17)

In [33]:
houses_predictions  = houses[['addr', 'living_space', 'space', 'area_num']]
houses_predictions['hs_ratio_pred'] = [0 for _ in range(houses_predictions.shape[0])]
houses_predictions['actual_space'] = [0 for _ in range(houses_predictions.shape[0])]
houses_predictions

/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_62733/859710355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_predictions['hs_ratio_pred'] = [0 for _ in range(houses_predictions.shape[0])]
/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_62733/859710355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_predictions['actual_space'] = [0 for _ in range(houses_predictions.shape[0])]


,addr,living_space,space,area_num,hs_ratio_pred,actual_space
0,"108809, Москва г, д. Толстопальцево, ул. Совет...",NaN,90.0,0,0,0
1,"108809, Москва г, д. Толстопальцево, ул. Совет...",NaN,211.0,0,0,0
2,"108809, Москва г, д. Толстопальцево, ул. Совет...",NaN,113.0,0,0,0
3,"108809, Москва г, д. Толстопальцево, ул. Совет...",NaN,275.0,0,0,0
4,"108809, Москва г, д. Толстопальцево, ул. Совет...",NaN,69.0,0,0,0
...,...,...,...,...,...,...
66795,"127055, Москва г, ул. Тихвинская, д. 39",16519.0,23595.3,97,0,0
66796,"127055, Москва г, ул. Тихвинская, д. 3, корп. 1",11495.0,13540.5,97,0,0
66797,"127055, Москва г, ул. Тихвинская, д. 7",1948.0,2652.6,97,0,0
66798,"127055, Москва г, ул. Тихвинская, д. 9",1241.0,5328.6,97,0,0


In [ ]:
i = 0
for area_name in houses_real_order.keys():
    n = houses_real_order[area_name]
    col_n = houses_dict[area_name]
    ratio = areas_geom[areas_geom['area_name'] == area_name]['hs_ratio']
    print(ratio.mean())
    for index, row in houses.iterrows():
        if row['area_num'] == col_n:
            sp = 0
            if not np.isnan(row['living_space']):
                sp = row['living_space']
            elif not np.isnan(row['space']):
                sp = row['space']
            if not np.isnan(row['living_space']) or not np.isnan(row['space']):
                houses_predictions.loc[houses_predictions["addr"] == row['addr'], "hs_ratio_pred"] = float(ratio.mean())
                i += 1

    print(n, i)
houses_predictions

6.879997569571029
0 810
21.44231803188166
1 810
15.292963338136014
2 1373
19.735399867331807
3 1582
16.196948573428376
4 1989
17.826088260538324
5 2098
21.6001101901392
6 2484
26.00066134043497
7 2668
25.77803619094916
8 2843
21.398498536072672
9 3043
22.559425329722274
10 3194
28.77772482705611
11 3525
18.301917506089932
12 3525
9.414699394522561
13 3733
19.53435278370562
14 3880
7.460302121650599
15 3880
16.879890446578578
16 3935
6.039997463252392
17 4230
22.31258883104726
18 4284
14.556436790560946
19 4556
18.24525179942563
20 4556
26.38308835810923
21 4556
16.23464397205292
22 5169
17.194418062700613
23 5391
8.01899641577061
24 5575
24.858392116301783
25 6006
23.883136862661047
26 6169
5.242274752523832
27 6394
10.005907480606467
28 6450
12.240409001843691
29 6541
9.40035595536789
30 6759
19.600438053391915
31 6883
13.45188568814287
32 7123
9.09655815882231
33 7407
17.4031708206687
34 7464
14.202236599168407
35 8483
16.00928670465732
36 8710
26.89490674318508
37 9183
18.2222269044

In [22]:
houses_predictions['hs_ratio_ppl'] = houses_predictions['actual_space'] * houses_predictions['hs_ratio_pred']

/var/folders/xq/qzhlj_357vs3kfkjsn1vg4rm0000gn/T/ipykernel_62733/3549127403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  houses_predictions['hs_ratio_ppl'] = houses_predictions['actual_space'] * houses_predictions['hs_ratio_pred']


In [23]:
area_predictions = areas_geom
houses_predictions.dropna(subset = ['hs_ratio_ppl'], inplace=True)
houses_predictions = houses_predictions[houses_predictions['hs_ratio_pred'] != 0]

houses_pred_aggr = houses_predictions[['area_num', 'hs_ratio_ppl']].groupby('area_num').sum()


area_predictions['hs_ratio_pred'] = houses_pred_aggr['hs_ratio_ppl']
area_predictions.dropna(subset = ['hs_ratio_pred'], inplace=True)
area_predictions

KeyError: ['slau_pred_ppl']

In [96]:
# area_pred_copy = area_predictions

In [119]:
import plotly.express as px
import plotly.graph_objects as go
area_predictions = area_predictions[area_predictions['area_name'] != 'Рязанский']
area_predictions = area_predictions[area_predictions['area_name'] != 'Тимирязевский']
area_predictions = area_predictions[area_predictions['area_name'] != 'Южное Тушино']
area_predictions = area_predictions.sort_values(by=['area_name'], ascending=False)

fig = go.Figure()
fig.add_scatter(x=area_predictions['area_name'], y=area_predictions['population'], name='actual')

fig.add_scatter(x=area_predictions['area_name'], y=area_predictions['hs_ratio_pred'], name='prediction through house_supply')
area_predictions['new_predict'] = area_predictions['hs_ratio_pred']*0.6 + area_predictions['population']*0.4
fig.add_scatter(x=area_predictions['area_name'], y=area_predictions['new_predict'], name='new_prediction')
fig.update_layout(title='Population prediction', xaxis_title='Area', yaxis_title='Population', height=800)
fig.show()

In [98]:
# houses_predictions.sample(n = 5)

In [99]:
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv
import matplotlib.pyplot as plt

In [100]:

houses

,addr,fias_code,lead_type,lead_name,type,cond,space,living_space,dorm_federal,dorm_municipal,dorm_type,room_type,demolition_date,geometry,lat,lon,area_num
0,"108809, Москва г, д. Толстопальцево, ул. Совет...",e2b601d1-9dce-4daa-93c2-633ee59f7f5c,Не выбран,NaN,Жилой,Исправный,90.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
1,"108809, Москва г, д. Толстопальцево, ул. Совет...",1389f7e1-1948-43c5-a769-944efdf1777d,Не выбран,NaN,Жилой,Исправный,211.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
2,"108809, Москва г, д. Толстопальцево, ул. Совет...",1bd548bd-d46a-4991-9062-57eeb638a2ce,Не выбран,NaN,Жилой,Исправный,113.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
3,"108809, Москва г, д. Толстопальцево, ул. Совет...",ac2010c3-0c83-47e6-b67e-ea555f181975,Не выбран,NaN,Жилой,Исправный,275.0,NaN,-,-,-,NaN,NaN,POINT (37.1928963 55.6036271),55.603627,37.192896,0
4,"108809, Москва г, д. Толстопальцево, ул. Совет...",55cc8c65-03a7-46f2-ae61-e8a6e3420b25,Не выбран,NaN,Жилой,Исправный,69.0,NaN,-,-,-,NaN,NaN,POINT (37.1898994 55.6071768),55.607177,37.189899,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66795,"127055, Москва г, ул. Тихвинская, д. 39",adf12acf-cfca-4b7e-acab-af962a58a5de,УО,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ТСЖ ...",Многоквартирный,Исправный,23595.3,16519.0,нет,нет,-,NaN,NaN,POINT (37.598968 55.791289),55.791289,37.598968,97
66796,"127055, Москва г, ул. Тихвинская, д. 3, корп. 1",01a3cfd9-65de-4712-b305-a8e9420a5576,УО,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,13540.5,11495.0,-,-,-,NaN,NaN,POINT (37.600882 55.785939),55.785939,37.600882,97
66797,"127055, Москва г, ул. Тихвинская, д. 7",7136e1b0-e4a6-44aa-a94d-ed4f956f561c,УО,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,2652.6,1948.0,-,-,-,NaN,NaN,POINT (37.600783 55.786779),55.786779,37.600783,97
66798,"127055, Москва г, ул. Тихвинская, д. 9",7ade45e2-1084-4c3a-8cc3-c4abc3129173,УО,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,5328.6,1241.0,-,-,-,NaN,NaN,POINT (37.600352 55.787275),55.787275,37.600352,97


In [101]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans
# from sklearn import metrics
# latitudes =  houses['lat'].values.reshape(-1, 1)
# longitudes =  houses['lon'].values.reshape(-1, 1)
#
# coordinates = np.column_stack((latitudes, longitudes))
#
# print(coordinates)
#
# # Define the range of possible cluster numbers to try
# k_values = range(2, 50)
#
# # Initialize a list to store the within-cluster sum of squares (inertia) for each k value
# inertia_values = []
#
# # Calculate the inertia for each k value
# for k in k_values:
#     kmeans = KMeans(n_clusters=k, random_state=0).fit(coordinates)
#     kmeans.fit(coordinates)
#     inertia_values.append(kmeans.inertia_)
#
# # Plot the elbow curve
# plt.plot(k_values, inertia_values, 'bx-')
# plt.xticks(np.arange(min(k_values), max(k_values)+1, 5.0))
# plt.yticks(np.arange(min(inertia_values), max(inertia_values)+1, 100.0))
# plt.xlabel('Number of Clusters (k)')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Geographic Coordinates')
# plt.show()

In [102]:
# houses

In [103]:
# cut_houses = houses
# df_features = pd.get_dummies(cut_houses['lead_type'], prefix='lead_type')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# df_features = pd.get_dummies(cut_houses['type'], prefix='type')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# df_features = pd.get_dummies(cut_houses['cond'], prefix='cond')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# df_features = pd.get_dummies(cut_houses['dorm_federal'], prefix='dorm_federal')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# df_features = pd.get_dummies(cut_houses['dorm_municipal'], prefix='dorm_municipal')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# df_features = pd.get_dummies(cut_houses['dorm_type'], prefix='dorm_type')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# df_features = pd.get_dummies(cut_houses['room_type'], prefix='room_type')
# cut_houses = pd.concat([cut_houses, df_features], axis=1)
#
# cols =  ['area_num', 'cond_Аварийный', 'cond_Ветхий',
#         'cond_Исправный', 'dorm_federal_-',
#         'dorm_federal_нет',  'dorm_municipal_-',
#         'dorm_municipal_да', 'dorm_municipal_нет',  'dorm_type_-',
#         'dorm_type_Гостиничный', 'dorm_type_Коридорный',
#          'lat', 'lead_type_ЖК',
#         'lead_type_Иной кооператив', 'lead_type_Не выбран',
#         'lead_type_Непосредственное управление', 'lead_type_ТСЖ',
#         'lead_type_УО', 'living_space', 'lon', 'room_type_Жилое',
#         'space',  'type_Жилой', 'type_Жилой дом блокированной застройки',
#         'type_Многоквартирный']
# cut_houses = cut_houses[cols]
# cut_houses['living_space'] = cut_houses['living_space'].fillna(0)
# cut_houses['space'] = cut_houses['living_space'].fillna(0)
#
# cut_houses

In [104]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans
# from sklearn import metrics
# latitudes =  houses['lat'].values.reshape(-1, 1)
# longitudes =  houses['lon'].values.reshape(-1, 1)
#
# coordinates = np.column_stack((latitudes, longitudes))
#
# print(coordinates)
#
# # Define the range of possible cluster numbers to try
# k_values = range(2, 10)
#
# # Initialize a list to store the within-cluster sum of squares (inertia) for each k value
# inertia_values = []
#
# # Calculate the inertia for each k value
# for k in k_values:
#     kmeans = KMeans(n_clusters=k, random_state=0).fit(cut_houses)
#     kmeans.fit(cut_houses)
#     inertia_values.append(kmeans.inertia_)
#
# # Plot the elbow curve
# plt.plot(k_values, inertia_values, 'bx-')
# plt.xticks(np.arange(min(k_values), max(k_values)+1, 5.0))
# plt.yticks(np.arange(min(inertia_values), max(inertia_values)+1, 100.0))
# plt.xlabel('Number of Clusters (k)')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Geographic Coordinates')
# plt.show()

In [105]:

# kmeans = KMeans(n_clusters = 7, init ='k-means++')
# kmeans.fit(cut_houses[cols]) # Compute k-means clustering.
# cut_houses['cluster_label'] = kmeans.fit_predict(cut_houses[cols])
# centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
# labels = kmeans.predict(cut_houses[cols]) # Labels of each point

# cut_houses.head(10)

In [106]:
# houses_geom = gpd.GeoDataFrame(
#     cut_houses, geometry=gpd.points_from_xy(cut_houses.lon, cut_houses.lat), crs="EPSG:4326")
# houses_geom.head()

In [107]:
# X = houses_geom.drop(columns=['geometry', 'cluster_label'])
#
# from sklearn.metrics import davies_bouldin_score, silhouette_score
# from sklearn.cluster import KMeans
#
# # Calculate the Davies-Bouldin Index
# dbi = davies_bouldin_score(X, houses_geom['cluster_label'])
# print("KMEANS - Davies-Bouldin Index:", dbi)
# silhouette_avg = silhouette_score(X, houses_geom['cluster_label'])
# print("KMEANS - Silhouette Score:", silhouette_avg)

In [108]:
# import plotly.express as px
# houses_geom["cluster_label"] = houses_geom["cluster_label"].astype(str)
#
# px.set_mapbox_access_token(open(".mapbox_token").read())
# fig = px.scatter_geo(houses_geom,
#                      lat=houses_geom.geometry.y,
#                      lon=houses_geom.geometry.x,
#                      color="cluster_label", # which column to use to set the color of markers
#                      projection="natural earth",
#                      size_max=0.2)
# fig.update_geos(fitbounds="locations")
# fig.update_layout(height=1000, margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
#
#


In [109]:
# import plotly.express as px
# px.set_mapbox_access_token(open(".mapbox_token").read())
# df = px.data.carshare()
# fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon",     color="peak_hour", size="car_hours",
#                         color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
# fig.show()

In [110]:
import hdbscan
import re


In [111]:
# model = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2,
#                         cluster_selection_epsilon=0.01)
# #min_cluster_size
# #min_samples
# #cluster_slection_epsilon
#
#
# class_predictions = model.fit_predict(cut_houses[cols])
# cut_houses['CLUSTER_HDBSCAN'] = class_predictions

In [112]:
cut_houses.to

AttributeError: 'DataFrame' object has no attribute 'to'

In [ ]:
# import plotly.express as px
# houses_geom["CLUSTER_HDBSCAN"] = houses_geom["CLUSTER_HDBSCAN"].astype(str)
#
# fig = px.scatter_geo(houses_geom,
#                      lat=houses_geom.geometry.y,
#                      lon=houses_geom.geometry.x,
#                      color="cluster_label", # which column to use to set the color of markers
#                      projection="natural earth")
# fig.update_geos(fitbounds="locations")
# fig.update_layout(height=8000, margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()


In [113]:
import geopandas as gpd
import plotly.express as px
import json
# geojson = {
#     "type": "FeatureCollection",
#     "crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::4283" } },
#     "features": [
#         { "type": "Feature", "properties": { "OBJECTID": 2282, "SORT_GBR_I": "20039", "LABEL_ID": "20-039", "SUB_NO": 100, "CODE": "20-039-100-106", "UNIQUE_ID": "20039100106", "FEATURE_C": 106, "GBR_NAME": "Hill Rock", "FEAT_NAME": "Rock", "QLD_NAME": "Hill Rock", "X_LABEL": "20-039S", "GBR_ID": "20039", "LOC_NAME_S": "Hill Rock (20-039)", "LOC_NAME_L": "Hill Rock (20-039)", "X_COORD": 148.90635681000001, "Y_COORD": -20.25865936, "Area_HA": 0.0, "GlobalID": "{6CFEA6F6-33A2-444A-835B-0003206EBA49}", "Shape_STAr": 14139.6796875, "Shape_STLe": 471.01884611600002 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 148.90694393116658, -20.259096202918855 ], [ 148.906921432860258, -20.259141198606248 ], [ 148.906754946292011, -20.259253691138628 ], [ 148.906687452271484, -20.259253691138628 ], [ 148.906655954642673, -20.259244690660317 ], [ 148.906619957352575, -20.259235691867001 ], [ 148.906570461078786, -20.259231191627361 ], [ 148.90651646604195, -20.259222192833235 ], [ 148.906302341814211, -20.259159602404967 ], [ 148.906223988958516, -20.259136699206643 ], [ 148.906196990990964, -20.259132199806906 ], [ 148.906156494039664, -20.259123201007046 ], [ 148.906124997309178, -20.259118700764201 ], [ 148.905985260773463, -20.259177555334624 ], [ 148.905936012434609, -20.259145698848446 ], [ 148.905900015144567, -20.259114202206661 ], [ 148.905882015601208, -20.259078205314452 ], [ 148.905873017177015, -20.259037707325927 ], [ 148.905864017854498, -20.258997211012321 ], [ 148.905859518193239, -20.258961213250245 ], [ 148.905850518870693, -20.258925217165245 ], [ 148.905850517972397, -20.258884719979569 ], [ 148.905850518870693, -20.258844222783306 ], [ 148.905854417559027, -20.25880628498949 ], [ 148.905859518193239, -20.25878122768551 ], [ 148.905859518193239, -20.258736231893799 ], [ 148.905850517972397, -20.258682236420832 ], [ 148.905859518193239, -20.258655238255969 ], [ 148.905850518870693, -20.258619241257328 ], [ 148.905850517972397, -20.258592243081473 ], [ 148.905850518870693, -20.258551745808919 ], [ 148.905864016956144, -20.258506749950669 ], [ 148.905868516617403, -20.258470752917578 ], [ 148.90586851841411, -20.258434755876166 ], [ 148.905873017177015, -20.258380760298351 ], [ 148.90588651616082, -20.258326764701771 ], [ 148.905891014923725, -20.25828626820272 ], [ 148.905900014246271, -20.25825476970104 ], [ 148.905904514805826, -20.258218772609524 ], [ 148.905922512552536, -20.25818277635242 ], [ 148.905922512552536, -20.258146779244225 ], [ 148.905998580094177, -20.258045355404413 ], [ 148.906057503288594, -20.257966792735232 ], [ 148.906106997765789, -20.257926296984994 ], [ 148.906165494260449, -20.257912796994663 ], [ 148.906205990313481, -20.257921796707411 ], [ 148.906232988281033, -20.257935295854242 ], [ 148.906273485232333, -20.257948794999852 ], [ 148.906295982640302, -20.257966792735221 ], [ 148.906462470106902, -20.258097282996108 ], [ 148.906493967735656, -20.258119780986227 ], [ 148.906516465143653, -20.258151278672489 ], [ 148.906547962772464, -20.258191775206793 ], [ 148.906570461977083, -20.258218772609524 ], [ 148.90659295938508, -20.258259269968963 ], [ 148.906624457013834, -20.258304265898925 ], [ 148.906646955320156, -20.258349262658626 ], [ 148.906750446630753, -20.258515749629034 ], [ 148.906790942683756, -20.258542746975422 ], [ 148.906831439635141, -20.258556246068217 ], [ 148.906871936586413, -20.258560746327401 ], [ 148.906898934553993, -20.258574243733147 ], [ 148.906943932064848, -20.258601242754846 ], [ 148.9069546902887, -20.258654134260489 ], [ 148.906948430827839, -20.258713733993034 ], [ 148.906934931844063, -20.258740732147746 ], [ 148.90692593341987, -20.258961214092974 ], [ 148.90694393116658, -20.259010710065613 ], [ 148.906948430827839, -20.259064706267022 ], [ 148.90694393116658, -20.259096202918855 ] ] ] } }]
# }
#
# gdf = gpd.GeoDataFrame.from_features(geojson)
point = (148.90635, -20.25866)

px.scatter_mapbox(lat=[point[1]], lon=[[0]]).update_layout(
    mapbox={
        "style": "open-street-map",
        "zoom": 16,
        "layers": [
            {
                "source": json.loads(gdf.geometry.to_json()),
                "below": "traces",
                "type": "line",
                "color": "purple",
                "line": {"width": 1.5},
            }
        ],
    },
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)
